In [1]:
import requests
from bs4 import BeautifulSoup
from Crypto.Cipher import AES
from Crypto.Protocol.KDF import PBKDF2
from Crypto.Util.Padding import pad, unpad
from binascii import unhexlify
import base64
import json
import urllib
from datetime import datetime

In [2]:
with open("data_ie.json", "r") as json_file:
    inp = json.load(json_file)

In [6]:
response0 = requests.request("GET", "https://www.dgft.gov.in/CP/")
awsalb=response0.headers['set-cookie'].split(";")[0]
html_content = response0.text
soup = BeautifulSoup(html_content, 'html.parser')
if csrf_value := soup.find('meta', attrs={'name': '_csrf'}):
    csrf = csrf_value.get('content')
    print("CSRF Content:", csrf)
else:
    print("CSRF Meta tag not found.")
    
headers = {
'Cookie':f'{awsalb}'}
payload=None

response0 = requests.request("POST", f"https://www.dgft.gov.in/CP/webHP?requestType=ApplicationRH&actionVal=visiterCount&screenId=90000802&_csrf={csrf}", headers=headers, data=payload)
awsalb=response0.headers['set-cookie'].split(";")[0]
jsessionId=response0.headers['set-cookie'].split(";")[-3].split(", ")[1]
headers = { 'Cookie':f'{jsessionId}; {awsalb}'}
payload=base64.b64decode("cG9ydGFsPUNBUw==")

response0 = requests.request("POST", f"https://www.dgft.gov.in/CP/webHP?requestType=ApplicationRH&actionVal=loginModel&screenId=90000512&_csrf={csrf}", headers=headers, data=payload)
awsalb=response0.headers['set-cookie'].split(";")[0]
html = response0.content
soup = BeautifulSoup(html, "html.parser")
desired_id_element = soup.find(id="temp_key")
salt = desired_id_element.get('value')
class AesUtil:
    def __init__(self, key_size, iteration_count):
        self.key_size = key_size // 8
        self.iteration_count = iteration_count

    def generate_key(self, salt, pass_phrase):
        return PBKDF2(
            pass_phrase.encode(),
            unhexlify(salt),
            dkLen=self.key_size,
            count=self.iteration_count,
        )

    def encrypt(self, salt, iv, pass_phrase, plain_text):
        key = self.generate_key(salt, pass_phrase)
        cipher = AES.new(key, AES.MODE_CBC, iv=unhexlify(iv))
        padded_text = pad(plain_text.encode(), AES.block_size)
        ciphertext = cipher.encrypt(padded_text)
        return base64.b64encode(ciphertext).decode()

    def decrypt(self, salt, iv, pass_phrase, cipher_text):
        key = self.generate_key(salt, pass_phrase)
        cipher = AES.new(key, AES.MODE_CBC, iv=unhexlify(iv))
        ciphertext = base64.b64decode(cipher_text)
        decrypted_data = unpad(cipher.decrypt(ciphertext), AES.block_size)
        return decrypted_data.decode()

def encrypt_password(plain_text, salt, iv, pass_phrase):
    iteration_count = 1000
    key_size = 128
    aes_util = AesUtil(key_size, iteration_count)
    return aes_util.encrypt(salt, iv, pass_phrase, plain_text)

def decrypt_password(cipher_text, salt, iv, pass_phrase):
    iteration_count = 1000
    key_size = 128
    aes_util = AesUtil(key_size, iteration_count)
    return aes_util.decrypt(salt, iv, pass_phrase, cipher_text)
plain_text = inp['password']
iv = "4d67fd8ee80132c6115e39880b08165d"
pass_phrase = "tcs@1234"
encrypted_result = encrypt_password(plain_text, salt, iv, pass_phrase)
print("Encrypted Password:", encrypted_result)
headers = {
'Cookie':f'{jsessionId}; {awsalb}'}
payload=base64.b64decode("cG9ydGFsPUNBUw==")

response0 = requests.request("POST", f"https://www.dgft.gov.in/CP/webHP?requestType=ApplicationRH&actionVal=commonCaptcha&screenId=90000512&_csrf={csrf}", headers=headers, data=payload)

html = response0.content
soup = BeautifulSoup(html, "html.parser")
captcha_element = soup.find(id="captcha")
captcha_id=captcha_element.get("src").split("?")[1]
awsalb=response0.headers['set-cookie'].split(";")[0]

headers = {
'Cookie':f'{jsessionId}; {awsalb}'}
payload=None

response0 = requests.request("GET", f"https://www.dgft.gov.in/CP/SimpleCaptcha?{captcha_id}", headers=headers, data=payload)
if response0.status_code == 200:
    image_path = "captcha_image3.jpg"
    with open(image_path, 'wb') as image_file:
        for chunk in response0.iter_content(chunk_size=128):
            image_file.write(chunk)
    print("CAPTCHA image saved as:", image_path)
else:
    print("Failed to fetch CAPTCHA image")
awsalb=response0.headers['set-cookie'].split(';')[0]

CSRF Content: b91e009b-cdc9-4ec5-bba0-e8278d3bad19
Encrypted Password: W+Kt1Vq3S6pgsOPRRH+2ZQ==
CAPTCHA image saved as: captcha_image3.jpg


In [7]:
captcha="KARP4"
headers = {
 'Cookie':f'{jsessionId}; {awsalb}'}
payload=None
response0 = requests.request("POST", f"https://www.dgft.gov.in/CP/webHP?requestType=ApplicationRH&actionVal=checkCaptcha&queryType=Select&screenId=114&captcha_val={captcha}&_csrf={csrf}", headers=headers, data=payload)
print(response0.text)
awsalb=response0.headers['set-cookie'].split(";")[0]

In [8]:
headers = {
 'Cookie':f'{jsessionId}; {awsalb}'}
data = {
    "username": inp['userid'],
    "password": encrypted_result,
    "txt_Captcha": captcha,
    "captcha_val": ""
}
response0 = requests.request("POST", "https://www.dgft.gov.in/CP/j_spring_security_check", headers=headers, data=data)
print(response0.status_code)
awsalb=response0.headers['set-cookie'].split(";")[0]
jsessionId=response0.headers['set-cookie'].split(";")[-4].split(', ')[1]
headers = {
 'Cookie':f'{jsessionId}; {awsalb}'}
payload=None

response0 = requests.request("GET", "https://www.dgft.gov.in/CP/web?requestType=ApplicationRH&actionVal=checkLogin", headers=headers, data=payload)
awsalb=response0.headers['set-cookie'].split(";")[0]
jsessionId=response0.headers['set-cookie'].split(";")[-4].split(', ')[1]
html_content = response0.text
soup = BeautifulSoup(html_content, 'html.parser')
if csrf_value := soup.find('meta', attrs={'name': '_csrf'}):
    csrf = csrf_value.get('content')
    print("CSRF Content:", csrf)
else:
    print("CSRF Meta tag not found.")
headers = {
 'Cookie':f'{jsessionId}; {awsalb}'}
payload=base64.b64decode("cG9ydGFsPUNBUw==")

response0 = requests.request("POST", f"https://www.dgft.gov.in/CP/web?requestType=ApplicationRH&menuCode=50001&actionVal=loadpage&screenId=50001&appId=201000001&_csrf={csrf}", headers=headers, data=payload)
awsalb=response0.headers['set-cookie'].split(";")[0]
jsessionId=response0.headers['set-cookie'].split(";")[-4].split(', ')[1]
headers = {
 'Cookie':f'{jsessionId}; {awsalb}'}
payload=None

response0 = requests.request("POST", f"https://www.dgft.gov.in/CP/web?requestType=ApplicationRH&actionVal=applicantDetailsFromRegistration&screenId=50001&_csrf={csrf}", headers=headers, data=payload)
awsalb=response0.headers['set-cookie'].split(";")[0]
jsessionId=response0.headers['set-cookie'].split(";")[-5].split(', ')[1]
headers = {
 'Cookie':f'{jsessionId}; {awsalb}'}
payload={
    "panNumber":inp['payload1']['iecRequestGeneralDetailsDTO']['panNumber']}

response0 = requests.request("POST", f"https://www.dgft.gov.in/CP/web?requestType=ApplicationRH&actionVal=isPanExist&screenId=50001&appId=2&_csrf={csrf}", headers=headers, data=payload)
awsalb=response0.headers['set-cookie'].split(";")[0]
jsessionId=response0.headers['set-cookie'].split(";")[-5].split(', ')[1]

headers = {
 'Cookie':f'{jsessionId}; {awsalb}'}
payload={
    "panNumber":inp['payload1']['iecRequestGeneralDetailsDTO']['panNumber'],
    "nameAsPan":inp['payload1']['iecRequestGeneralDetailsDTO']['nameAsPan'],
    "birthdate":inp['payload1']['iecRequestGeneralDetailsDTO']['birthdate'],
    "panChange":"true"
}
response0 = requests.request("POST", f"https://www.dgft.gov.in/CP/web?requestType=ApplicationRH&actionVal=validatePan&screenId=50001&_csrf={csrf}", headers=headers, data=payload)
awsalb=response0.headers['set-cookie'].split(";")[0]
jsessionId=response0.headers['set-cookie'].split(";")[-5].split(', ')[1]

headers = {
 'Cookie':f'{jsessionId}; {awsalb}'}
payload = {
    "iecModule[iecRequestGeneralDetailsDTO][cin]": "undefined",
    "cin": inp['payload1']['iecRequestGeneralDetailsDTO']['cin'],
    "gstin": ""
}
response0 = requests.request("POST", f"https://www.dgft.gov.in/CP/web?requestType=ApplicationRH&actionVal=getCINData&screenId=50001&_csrf={csrf}", headers=headers, data=payload)
awsalb=response0.headers['set-cookie'].split(";")[0]
jsessionId=response0.headers['set-cookie'].split(";")[-5].split(', ')[1]


200
CSRF Content: 942cf958-79fa-4a04-94fc-e7d5dfd6b5b0


In [12]:
inp['payload1']['iecRequestGeneralDetailsDTO']['isCINChange']=json.loads(response0.text)['isCINChange']

In [13]:
if json.loads(response0.text)['sezEntity'] == None:
    inp['payload1']['iecRequestGeneralDetailsDTO']['sezEntity']='NO'
else:
    inp['payload1']['iecRequestGeneralDetailsDTO']['sezEntity']= json.loads(response0.text)['sezEntity']
if json.loads(response0.text)['sezEntity'] == None:
    inp['payload1']['iecRequestGeneralDetailsDTO']['eouEntity']='NO'
else:
    inp['payload1']['iecRequestGeneralDetailsDTO']['eouEntity']= json.loads(response0.text)['eouEntity']

In [14]:
inp['generalAttach']

'https://legalnitiai.s3.ap-south-1.amazonaws.com/GST-reg/Certificate+of+Incorporation+.pdf'

In [15]:

headers = {
    'Cookie': f'{jsessionId}; {awsalb}'
}

file_url1 = r'''https://legalnitiai.s3.ap-south-1.amazonaws.com/GST-reg/Certificate+of+Incorporation+.pdf'''
filename1=file_url1.split('/')[-1].replace("+"," ")
response1 = requests.get(file_url1)
inp['payload1']['iecRequestGeneralDetailsDTO']['totalFileSize_generalAttach']=str(int(response1.headers['Content-length'])/(1024*1024))[:4]
if response1.status_code == 200:
    files = {
        'importFilegeneralAttach': (filename1, response1.content, 'application/pdf')
    }
    response0 = requests.post(f"https://www.dgft.gov.in/CP/Upload?attachmentNameHidden=generalAttach&customFunction=&_csrf={csrf}&attachment_type=undefined&_csrf={csrf}", headers=headers, files=files)
    if response0.status_code == 200:
        print("Request was successful.")
    else:
        print(f"Request failed with status code: {response0.status_code}")
else:
    print(f"Failed to download file with status code: {response0.status_code}")
soup = BeautifulSoup(response0.text, 'html.parser')
attachment_input = soup.find('input', {'id': 'AttachmentString_generalAttach'})
gen_attach=attachment_input.get('value')
print(gen_attach)
awsalb=response0.headers['set-cookie'].split(";")[0]

Request was successful.
2023/09/14/1694636153679_1415824444171480273.tmp* *Certificate of Incorporation .pdf*  *0


In [16]:
file_url1.split('/')[-1].replace("+"," ")

'Certificate of Incorporation .pdf'

In [17]:
headers = {
 'Cookie':f'{jsessionId}; {awsalb}'}
payload=None
response0 = requests.request("POST", f"https://www.dgft.gov.in/CP/web?requestType=ApplicationRH&actionVal=pincodeData&screenId=99000888&pincode={inp['payload1']['iecRequestGeneralDetailsDTO']['pincode']}&_csrf={csrf}", headers=headers)
awsalb=response0.headers['set-cookie'].split(";")[0]
jsessionId=response0.headers['set-cookie'].split(";")[-5].split(', ')[1]

In [18]:
json.loads(response0.text)

{'state': {'key': 29, 'value': 'KARNATAKA'},
 'district': {'key': 273, 'value': 'BENGALURU URBAN'},
 'dgft': {'key': None, 'value': None},
 'districts': [],
 'pincode': '560004'}

In [19]:
inp

{'payload1': {'iecRequestDirectorDetailsDTOs': [],
  'modifyReqType': 'Update PAN Details',
  'iecRequestGeneralDetailsDTO': {'deficiency': '',
   'natureOfConcern': {'key': '111000113', 'value': 'Private Limited'},
   'entityName': 'BB FINERY PRIVATE LIMITED',
   'panNumber': 'AALCB5105H',
   'nameAsPan': 'BB FINERY PRIVATE LIMITED',
   'birthdate': '12/04/2023',
   'preferredActivity': {'key': '111000121', 'value': 'Merchant Exporter'},
   'sezEntity': 'NO',
   'eouEntity': 'NO',
   'cin': 'U82990KA2023PTC172277',
   'gstin': '',
   'addMobile': '7050541423',
   'mobOrEmailModified': 'false',
   'addEmail': 'Satvikmishra2615@gmail.com',
   'otpMobile': '',
   'otpEmail': '',
   'pleaseSpecifyNatureOfConcern': '',
   'pleaseSpecifyPreferred': '',
   'AttachmentString_generalAttach': '2023/09/01/1693548532247_2480511743773392403.tmp* *Part B statement (2).pdf* *0',
   'formId': 'addForm',
   'customFunction': '',
   'attachmentName': 'addressAttach',
   'attachmentComponentHeader_gener

In [20]:
inp['payload1']['iecRequestGeneralDetailsDTO']['state']=json.loads(response0.text)['state']
inp['payload1']['iecRequestGeneralDetailsDTO']['district']=json.loads(response0.text)['district']

In [21]:
headers = {
 'Cookie':f'{jsessionId}; {awsalb}'}
payload= {
    "pincode": inp['payload1']['iecRequestGeneralDetailsDTO']['pincode'],
    "districtkey": inp['payload1']['iecRequestGeneralDetailsDTO']['district']['key'],
    "districtvalue": inp['payload1']['iecRequestGeneralDetailsDTO']['district']['value'],
    "sezEntity": inp['payload1']['iecRequestGeneralDetailsDTO']['sezEntity']
}
response0 = requests.request("POST", f"https://www.dgft.gov.in/CP/web?requestType=ApplicationRH&actionVal=getDGFTOffice&screenId=50001&_csrf={csrf}", headers=headers, data=payload)
awsalb=response0.headers['set-cookie'].split(";")[0]
jsessionId=response0.headers['set-cookie'].split(";")[-5].split(', ')[1]

In [22]:
inp['payload1']['iecRequestGeneralDetailsDTO']['dgft']=json.loads(response0.text)[0]

In [23]:

headers = {
    'Cookie': f'{jsessionId}; {awsalb}'
}

file_url2 = inp['addressAttach']
filename2=file_url2.split('/')[-1].replace("+"," ")
response2 = requests.get(file_url2)
inp['payload1']['iecRequestGeneralDetailsDTO']['totalFileSize_addressAttach']=totalFileSize_addressAttach=str(int(response1.headers['Content-length'])/(1024*1024))[:4]

if response2.status_code == 200:
    files = {
        'importFileaddressAttach': (filename2, response2.content, 'application/pdf')
    }
    response0 = requests.post(f"https://www.dgft.gov.in/CP/Upload?attachmentNameHidden=addressAttach&customFunction=&_csrf={csrf}&attachment_type=undefined&_csrf={csrf}", headers=headers, files=files)
    if response0.status_code == 200:
        print("Request was successful.")
    else:
        print(f"Request failed with status code: {response0.status_code}")
else:
    print(f"Failed to download file with status code: {response0.status_code}")
soup = BeautifulSoup(response0.text, 'html.parser')
if attachment_input := soup.find(
    'input', {'id': 'AttachmentString_addressAttach'}
):
    Address_attach = attachment_input.get('value')
    print("Value of ADDRESSattach:", Address_attach)
else:
    print("AttachmentString_generalAttach not found in the response.")
awsalb=response0.headers['set-cookie'].split(";")[0]

Request was successful.
Value of ADDRESSattach: 2023/09/14/1694636157754_942242241562998513.tmp* *Electricity.pdf*  *0


In [24]:
inp['payload1']['iecRequestGeneralDetailsDTO']['AttachmentString_generalAttach']=gen_attach
inp['payload1']['iecRequestGeneralDetailsDTO']['AttachmentString_addressAttach']=Address_attach
inp['payload1']['iecRequestGeneralDetailsDTO']['generalDetailsAttachmentDTO']['s3Path']=gen_attach
inp['payload1']['iecRequestGeneralDetailsDTO']['addressDetailsAttachmentDTO']['s3Path']=Address_attach
payload={
    "iecModule":json.dumps(inp['payload1']),
    "draftId":"",
    "panNumber":inp['payload1']['iecRequestGeneralDetailsDTO']['panNumber'],
    "nameAsPan":inp['payload1']['iecRequestGeneralDetailsDTO']['panNumber'],
    "birthdate":inp['payload1']['iecRequestGeneralDetailsDTO']['birthdate']
}
headers = {
 'Cookie':f'{jsessionId}; {awsalb}'}
response3 = requests.request("POST", f"https://www.dgft.gov.in/CP/web?requestType=ApplicationRH&actionVal=handleRequest&screenId=50001&direction=next&currentTab=generalInformation&fromTab=&fromIndex=&toIndex=&_csrf={csrf}", headers=headers, data=payload)
soup = BeautifulSoup(response3.text, 'html.parser')
app_no=soup.find(id="applicationNumber").get('value')
draftId=soup.find(id="draftId").get("value")
awsalb=response3.headers['set-cookie'].split(";")[0]
jsessionId=response3.headers['set-cookie'].split(";")[-4].split(', ')[1]
jsessionId

'JSESSIONID=B6E049EC44CFEBB4C4280F61080A002A'

In [25]:
a=json.loads(soup.find(id="moduleJson").get("value"))

In [26]:
a

{'deficiencyDto': {'application_no': None,
  'req_file_no': None,
  'deficiency_issued_date': None,
  'submited_date': None,
  'page': None,
  'deficiencyUrl': None,
  'req_id': None,
  'deficiency': False,
  'remarks': None,
  'attachedBy': None,
  'appId': 201000001},
 'iecRequestAddressDetailsDTO': None,
 'iecRequestBranchDetailsDTOs': None,
 'bankDetailsDTOs': [],
 'iecRequestBankDetailsDTO': None,
 'iecRequestOtherDetailsDTO': None,
 'iecRequestDirectorDetailsDTOs': [{'siNo': None,
   'din': '0009730658',
   'dinInt': '9730658',
   'ifForeignDirector': None,
   'passportNumber': None,
   'directorId': None,
   'iecDirectorId': None,
   'panDirector': None,
   'fatherName': None,
   'birthdateDir': None,
   'addressLine1': None,
   'addressLine2': None,
   'city': None,
   'directorState_key': None,
   'title_key': None,
   'title_Value': None,
   'directorState_Value': None,
   'directorDistrict_Value': None,
   'directorDistrict_key': None,
   'mcaValidated': 'true',
   'title': 

In [27]:
a['iecRequestDirectorDetailsDTOs']=inp['iecRequestDirectorDetailsDTOs']

In [28]:
payload={
    "iecModule":json.dumps(a),
    "draftId":draftId,
    "din":"",
    "directorNameAsPan":"",
    "panDirector":"",
    "birthdateDir":""
}
headers = {
 'Cookie':f'{jsessionId}; {awsalb}'}
response3 = requests.request("POST", f"https://www.dgft.gov.in/CP/web?requestType=ApplicationRH&actionVal=handleRequest&screenId=50001&direction=next&currentTab=detailsOfProperioter&fromTab=&fromIndex=&toIndex=&_csrf={csrf}", headers=headers, data=payload)
soup = BeautifulSoup(response3.text, 'html.parser')
awsalb=response3.headers['set-cookie'].split(";")[0]
jsessionId=response3.headers['set-cookie'].split(";")[-4].split(', ')[1]
jsessionId

'JSESSIONID=B6E049EC44CFEBB4C4280F61080A002A'

In [29]:
b=json.loads(soup.find(id="moduleJson").get("value"))

In [30]:

headers = {
    'Cookie': f'{jsessionId}; {awsalb}'
}

file_url3 = inp['importFile1']
filename3=file_url3.split('/')[-1].replace("+"," ")
response3 = requests.get(file_url3)
# inp['iecRequestGeneralDetailsDTO']['totalFileSize_addressAttach']=str(int(response1.headers['Content-length'])/(1024*1024))[:4]

if response3.status_code == 200:
    files = {
        'importFile1': (filename3, response3.content, 'application/pdf')
    }
    response0 = requests.post(f"https://www.dgft.gov.in/CP/Upload?attachmentNameHidden=1&attachmentNameSingle=attach_bank&attachmentFormName=bankForm&_csrf={csrf}&reDirectPage=singleDgftAttachment&dgftSingleAllowedExtensions=pdf&_csrf={csrf}", headers=headers, files=files)
    if response0.status_code == 200:
        print("Request was successful.")
    else:
        print(f"Request failed with status code: {response0.status_code}")
else:
    print(f"Failed to download file with status code: {response0.status_code}")
soup = BeautifulSoup(response0.text, 'html.parser')
if attachmentStr := soup.find(
    'input', {'id': 'attachmentStr'}
):
    attachmentStr = attachment_input.get('value')
    print("Value of attachmentStr:", attachmentStr)
else:
    print("attachmentStr not found in the response.")
awsalb=response0.headers['set-cookie'].split(";")[0]

Request was successful.
Value of attachmentStr: 2023/09/14/1694636157754_942242241562998513.tmp* *Electricity.pdf*  *0


In [31]:
b['bankDetailsDTOs']=inp['bankDetailsDTOs']
# b['iecRequestBankDetailsDTO']={'accNumber': '',
#  'accHolder': '',
#  'ifsCode': '',
#  'bankNameKeyValueDto': {'key': '-1', 'value': 'Please Select'},
#  'isMandatory': '1',
#  'bankName': 'IDFC BANK',
#  'branchAddress': '',
#  'aswS3Path': '',
#  'fileName': '',
#  'attachmentType': '101',
#  'attachmentStr': '',
#  'dgftSingleAllowedExtensions': 'pdf',
#  'maxFileSize': '',
#  'maxFileSizeMsg': '',
#  'bankValidRms': 'false',
#  'bankAttachmentDTO': {}}

In [32]:
b['bankDetailsDTOs'][0]['bankAttachmentDTO']['s3Path']=attachmentStr
b['bankDetailsDTOs'][0]['bankAttachmentDTO']['fileName']=filename3

In [33]:
payload={
    "iecModule":json.dumps(b),
    "draftId":draftId,
    "bankName":b['bankDetailsDTOs'][0]['bankName'],
    "branchAddress":"",
    "accHolder":"",
    "ifsCode":""
}
headers = {
 'Cookie':f'{jsessionId}; {awsalb}'}
response3 = requests.request("POST", f"https://www.dgft.gov.in/CP/web?requestType=ApplicationRH&actionVal=handleRequest&screenId=50001&direction=next&currentTab=bankInformation&fromTab=&fromIndex=&toIndex=&_csrf={csrf}", headers=headers, data=payload)


In [34]:
soup = BeautifulSoup(response3.text, 'html.parser')
awsalb=response3.headers['set-cookie'].split(";")[0]
jsessionId=response3.headers['set-cookie'].split(";")[-4].split(', ')[1]
jsessionId

'JSESSIONID=B6E049EC44CFEBB4C4280F61080A002A'

In [35]:
c=json.loads(soup.find(id="moduleJson").get("value"))

In [36]:
c

{'deficiencyDto': {'application_no': None,
  'req_file_no': None,
  'deficiency_issued_date': None,
  'submited_date': None,
  'page': None,
  'deficiencyUrl': None,
  'req_id': None,
  'deficiency': False,
  'remarks': None,
  'attachedBy': None,
  'appId': 201000001},
 'iecRequestAddressDetailsDTO': None,
 'iecRequestBranchDetailsDTOs': None,
 'bankDetailsDTOs': [{'faxnumber': None,
   'isNewBankDetails': None,
   'isMandatory': None,
   'bankNameKeyValueDto': {'key': 0, 'value': 'IDFC BANK'},
   'bankName': 'IDFC BANK',
   'branchAddress': 'Cunningham',
   'accNumber': '10107749364',
   'accHolder': 'BB FINERY',
   'ifsCode': 'IDFB0080176',
   'json': None,
   'bankFlag': None,
   'bankValidRms': None,
   'bankAttachmentDTO': {'attachId': None,
    's3Path': '2023/09/14/1694636157754_942242241562998513.tmp* *Electricity.pdf*  *0',
    'fileName': 'cancelled cheque.pdf',
    'fileType': None,
    'fileSize': None,
    'attachmentType': {'key': 10000002, 'value': 'IEC Bank DOC'},
    

In [37]:
print(response3.text)







	
	
	<input type="hidden" id="currentTabJson" value="{&quot;tabName&quot;:&quot;Other Details (Preferred sectors of operations)&quot;,&quot;viewName&quot;:&quot;otherDetails&quot;,&quot;moduleName&quot;:&quot;otherDetails&quot;,&quot;isVisited&quot;:false}" >
	<input type="hidden" id="moduleJson" value="{&quot;deficiencyDto&quot;:{&quot;application_no&quot;:null,&quot;req_file_no&quot;:null,&quot;deficiency_issued_date&quot;:null,&quot;submited_date&quot;:null,&quot;page&quot;:null,&quot;deficiencyUrl&quot;:null,&quot;req_id&quot;:null,&quot;deficiency&quot;:false,&quot;remarks&quot;:null,&quot;attachedBy&quot;:null,&quot;appId&quot;:201000001},&quot;iecRequestAddressDetailsDTO&quot;:null,&quot;iecRequestBranchDetailsDTOs&quot;:null,&quot;bankDetailsDTOs&quot;:[{&quot;faxnumber&quot;:null,&quot;isNewBankDetails&quot;:null,&quot;isMandatory&quot;:null,&quot;bankNameKeyValueDto&quot;:{&quot;key&quot;:0,&quot;value&quot;:&quot;IDFC BANK&quot;},&quot;bankName&quot;:&quot;IDFC BANK&qu

In [38]:
checkboxes = soup.find_all('input', {'name': 'importlist'})

# Create a dictionary to store the text and their values
text_and_values = {}

# Iterate through the checkboxes and extract text and values
for checkbox in checkboxes:
    text = checkbox['data-text']
    value = checkbox['value']
    text_and_values[text] = value

In [39]:
txt="Other"

In [40]:
matching_input = soup.find_all('input', {'data-text': txt})
matching_input

[]

In [41]:
print(response3.text)







	
	
	<input type="hidden" id="currentTabJson" value="{&quot;tabName&quot;:&quot;Other Details (Preferred sectors of operations)&quot;,&quot;viewName&quot;:&quot;otherDetails&quot;,&quot;moduleName&quot;:&quot;otherDetails&quot;,&quot;isVisited&quot;:false}" >
	<input type="hidden" id="moduleJson" value="{&quot;deficiencyDto&quot;:{&quot;application_no&quot;:null,&quot;req_file_no&quot;:null,&quot;deficiency_issued_date&quot;:null,&quot;submited_date&quot;:null,&quot;page&quot;:null,&quot;deficiencyUrl&quot;:null,&quot;req_id&quot;:null,&quot;deficiency&quot;:false,&quot;remarks&quot;:null,&quot;attachedBy&quot;:null,&quot;appId&quot;:201000001},&quot;iecRequestAddressDetailsDTO&quot;:null,&quot;iecRequestBranchDetailsDTOs&quot;:null,&quot;bankDetailsDTOs&quot;:[{&quot;faxnumber&quot;:null,&quot;isNewBankDetails&quot;:null,&quot;isMandatory&quot;:null,&quot;bankNameKeyValueDto&quot;:{&quot;key&quot;:0,&quot;value&quot;:&quot;IDFC BANK&quot;},&quot;bankName&quot;:&quot;IDFC BANK&qu

In [42]:
c['iecRequestOtherDetailsDTO']=inp['iecRequestOtherDetailsDTO']

In [43]:
c['iecRequestOtherDetailsDTO']

{'importList': [{'key': '111000130', 'value': 'Chemicals or Allied Products'},
  {'key': '111000809', 'value': 'Arms & ammunition'}],
 'importServicesList': [{'key': '111000600',
   'value': 'Information Technology (IT & ITES) Services'}],
 'exportList': [{'key': '111000148', 'value': 'Chemicals or Allied Products'}],
 'exportServicesList': [{'key': '111000724',
   'value': 'Wholesale trade services'},
  {'key': '111000725', 'value': 'Other Services'}],
 'exportServices': {'key': '111000146', 'value': 'Services'},
 'exportGoodsOtherSpecify': '',
 'exportGoodsOther': False,
 'importGoodsOtherSpecify': 'asdasda',
 'importGoodsOther': True,
 'please_specify': '',
 'isImport': True,
 'isExport': True,
 'isOther': False,
 'isGoods': True,
 'isServices': True,
 'importGoods': True,
 'importServices': True}

In [44]:
payload={
    "iecModule":json.dumps(c),
    "draftId":draftId
}
headers = {
 'Cookie':f'{jsessionId}; {awsalb}'}
response3 = requests.request("POST", f"https://www.dgft.gov.in/CP/web?requestType=ApplicationRH&actionVal=handleRequest&screenId=50001&direction=next&currentTab=otherDetails&fromTab=&fromIndex=&toIndex=&_csrf={csrf}", headers=headers, data=payload)


In [45]:
soup = BeautifulSoup(response3.text, 'html.parser')
awsalb=response3.headers['set-cookie'].split(";")[0]
jsessionId=response3.headers['set-cookie'].split(";")[-4].split(', ')[1]
jsessionId

'JSESSIONID=B6E049EC44CFEBB4C4280F61080A002A'

In [46]:
d=json.loads(soup.find(id="moduleJson").get("value"))

In [47]:
d['place']=inp['place']
d['declarationDate']=datetime.now().strftime("%d-%m-%Y")
d['declarationCheck']=True

In [48]:
payload={
    "iecModule":json.dumps(d),
     "draftId":draftId,
     "place":d['place'],
     "appId":"1"
}
headers = {
 'Cookie':f'{jsessionId}; {awsalb}'}
response3 = requests.request("POST", f"https://www.dgft.gov.in/CP/web?requestType=ApplicationRH&actionVal=handleRequest&screenId=50001&direction=next&currentTab=declaration&fromTab=&fromIndex=&toIndex=&_csrf={csrf}", headers=headers, data=payload)


In [49]:
print(response3.text)





<div class="modal" id="IECTermsModal" role="dialog">
	<div class="modal-dialog modal-lg modal-dialog-centered">
		<div class="modal-content">
			<!-- Modal body -->
			<div class="modal-body p-0">
				<div class="row pt-0">
					<div class="col-md-12 text-right">
						<button type="button" class="close mr-2 mt-1" data-dismiss="modal">&times;</button>
					</div>
				</div>
				<div class="row">
					<div class="col-md-12">
						<!-- 	<button type="button" class="close px-2 custom-close" data-dismiss="modal">&times;</button> -->
						<div
							class="main-section flex-column h-100 align-items-center justify-content-between d-flex">
							<h3
								class="font-weight-semi-bold font-28 text-center text-theme pt-5 preHide1">Declaration</h3>
							<div class="row">
								<div class="bg-white custom-declaration">
									<ul class="declaration-number">
										<li>I/We hereby certify that :</li>
										<ul class="declaration-alphabet pl-0">
											<li>The Firm for wh